<a href="https://colab.research.google.com/github/jong9810/TensorFlow-2.0/blob/main/11_Callback_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11. TensorFlow Callback Function

## Callback 개념

- 알람시계와 비슷한 원리
1. 먼저 특정한 상황에서 실행되는 함수를 시스템에 등록해 둔다.
2. 해당 상황이 발생했을 때 등록되어 있는 함수가 실행 되고
3. 시스템에서의 결과를 통해서 개발자는 등록된 콜백 함수가 실행된 것을 알 수 있다.

## TensorFlow Callback Function

- TensorFlow에는 다양한 콜백 함수가 있는데, 모델의 학습 방향, 저장 시점 그리고 학습 정지 시점 등에 관한 상황을 모니터링하기 위해 주로 사용됨
- 학습 도중에 학습율을 변화시키거나, 일정 시간이 지나도 검증 데이터 손실 값이 개선되지 않으면 학습을 멈추게 하는 등의 작업을 수행함
1. ReduceLPOnPlateau : 학습 중에 학습율을 변화시키는 함수
2. ModelCheckpoint : 모델의 가중치값을 중간에 저장하도록 하는 함수
3. EarlyStopping : 모델 성능 지표가 일정 시간동안 개선되지 않을 때 조기 종료하도록 하는 함수

### 1. ReduceLROnPlateau
- 모델의 성능 개선이 없을 경우, 학습율을 조절해서 모델의 개선을 유도하는 콜백 함수
1. 파라미터
- monitor : 무엇을 기준으로 함수를 실행할 지 지정
- factor : 통해 학습율을 얼만큼 줄일지 지정
- patience : 얼마만큼 epoch 동안 개선이 안될 시 함수 실행할 지 지정
- verbose : 로그를 출력할 형식을 지정
2. 예시
- reduceLR = ReduceLROnPlateau(monitor='val_loss', factpr=0.5, patience=5, verbose=1)
- hist = model.fit(x_train, t_train, epochs=50, validation_split=0.2, callbacks=[reduceLR])
- callbacks=[] : 콜백 함수를 시스템에 지정

### 2. ModelCheckpoint
- 모델의 학습 중에 조건을 만족했을 때 현재의 가중치를 중간 저장해줌
- 이점 : 모델의 학습 시간이 엄청나게 오래 걸리는 경우, 중간에 memory overflow나 crush가 나더라도 다시 가중치를 불러와서 학습을 이어나갈 수 있음. => 학습시간 단축
1. 파라미터
- file_path : 저장할 파일의 경로
- monitor : 어떤 값이 개선되었을 때 호출할 지 지정
- verbose : 로그 출력 형식을 지정
- save_best_only : best 값만을 저장할 것인지 결정
- mode : 만약 'auto'라면 자동으로 best 값을 찾음
2. 예시
- file_path = './modelchpoint_test.h5'
- checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
- hist = model.fit(x_train, t_train, epochs=50, validation_split=0.2, callbacks=[checkpoint])



### 3. EarlyStopping
- 모델 성능 지표가 우리가 설정한 epoch 동안 개선되지 않으면 조기 종료함
- 일반적으로 EarlyStoppin과 ModelCheckpoint를 조합해서 개선되지 않는 학습에 대해 조기 종료를 하고 ModelCheckpoint로 best model을 다시 로드하여 학습을 재게한다.
1. 파라미터
- monitor : 어떤 값이 개선되지 않을 때 함수를 호출할 지 지정
- patience : 몇 번의 epoch 동안 개선되지 않으면 함수를 호출할 지 지정
2. 예시
- file_path = './modelchpoint_test.h5'
- checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
- stopping = EarlyStopping(monitor='val_loss', patience=5)
- hist = model.fit(x_train, t_train, validation_split=0.2, callbacks=[checkpoint, stopping])

## Callback Function 예제 - MNIST

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
tf.__version__

In [ ]:
# 1. 데이터 불러오기
(x_train, t_train), (x_test, t_test) = mnist.load_data()

# print(x_train.shape, t_train.shape)
# print(x_test.shape, t_test.shape)

In [ ]:
# print(t_train)
# print(t_test)

In [ ]:
plt.figure(figsize=(6,6))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.axis('off')

plt.show()

In [ ]:
# 2. 데이터 전처리 과정
# 이미지 데이터 정규화
x_train = x_train / 255.0
x_test = x_test / 255.0

# 정답 데이터 원핫 인코딩
t_train = tf.keras.utils.to_categorical(t_train, num_classes=10)
t_test = tf.keras.utils.to_categorical(t_test, num_classes=10)

In [ ]:
# print(t_train)
# print(t_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=(28,28))) # 28 * 28 크기의 2차원 이미지를 784개의 1차원 벡터로 변환
model.add(Dense(100, activation='relu')) # 은닉층 (활성화 함수 : 'relu', 노드 수 : 100개)
model.add(Dense(10, activation='softmax')) # 출력층 (활성화 함수 : 'softmax', 노드 수(=정답 수) : 10개)

# 원핫 인코딩 방식이므로 loss는 'categorical_crossentropy'
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

file_path = './modelchpoint_test.h5'

checkpoint = ModelCheckpoint(file_path, monitor='val_loss', save_best_only=True, mode='auto', verbose=1)
stopping = EarlyStopping(monitor='val_loss', patience=5)

hist = model.fit(x_train, t_train, epochs=30, validation_split=0.2, callbacks=[checkpoint, stopping])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

hist = model.fit(x_train, t_train, epochs=30, validation_split=0.2, callbacks=[reduceLR])